In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sbn
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

dataset = pd.read_csv("dataset.csv", sep=';', decimal=',')

dataset

,Hour (Coded),Immobilized bus,Broken Truck,Vehicle excess,Accident victim,Running over,Fire vehicles,Occurrence involving freight,Incident involving dangerous freight,Lack of electricity,Fire,Point of flooding,Manifestations,Defect in the network of trolleybuses,Tree on the road,Semaphore off,Intermittent Semaphore,Slowness in traffic (%)
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.1
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.6
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.7
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9.2
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,23,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,17.8
131,24,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,18.1
132,25,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,17.7
133,26,0,4,0,0,0,0,0,0,0,0,0,1,0,0,1,0,17.4


In [65]:
dataset.describe()

,Hour (Coded),Immobilized bus,Broken Truck,Vehicle excess,Accident victim,Running over,Fire vehicles,Occurrence involving freight,Incident involving dangerous freight,Lack of electricity,Fire,Point of flooding,Manifestations,Defect in the network of trolleybuses,Tree on the road,Semaphore off,Intermittent Semaphore,Slowness in traffic (%)
count,135.00000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000
mean,14.00000,0.340741,0.874074,0.029630,0.422222,0.118519,0.007407,0.007407,0.007407,0.118519,0.007407,0.118519,0.051852,0.229630,0.044444,0.125926,0.014815,10.051852
std,7.81789,0.659749,1.102437,0.170195,0.696116,0.346665,0.086066,0.086066,0.086066,0.504485,0.086066,0.712907,0.222554,0.818998,0.206848,0.464077,0.121261,4.363243
min,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.400000
25%,7.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.400000
50%,14.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000
75%,21.00000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.850000
max,27.00000,4.000000,5.000000,1.000000,3.000000,2.000000,1.000000,1.000000,1.000000,4.000000,1.000000,7.000000,1.000000,8.000000,1.000000,4.000000,1.000000,23.400000


In [66]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 18 columns):
Hour (Coded)                             135 non-null int64
Immobilized bus                          135 non-null int64
Broken Truck                             135 non-null int64
Vehicle excess                           135 non-null int64
Accident victim                          135 non-null int64
Running over                             135 non-null int64
Fire vehicles                            135 non-null int64
Occurrence involving freight             135 non-null int64
Incident involving dangerous freight     135 non-null int64
Lack of electricity                      135 non-null int64
Fire                                     135 non-null int64
Point of flooding                        135 non-null int64
Manifestations                           135 non-null int64
Defect in the network of trolleybuses    135 non-null int64
Tree on the road                         135 non-null i

In [67]:
# Podzial data setu na Cel i Dane
X = dataset.iloc[:,  :-1].values
y = dataset.iloc[:, -1:].values

In [74]:
# Podział na dane treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

[splitting](https://towardsdatascience.com/6-amateur-mistakes-ive-made-working-with-train-test-splits-916fabb421bb)

In [75]:
pd.DataFrame(y_train).head()

,0
0,10.1
1,6.5
2,5.3
3,8.2
4,17.4


In [76]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.0,101.0,101.000000,101.000000,101.0,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000
mean,13.762376,0.346535,0.841584,0.019802,0.405941,0.118812,0.0,0.0,0.009901,0.148515,0.0,0.148515,0.059406,0.267327,0.049505,0.138614,0.009901
std,7.811720,0.684626,1.083814,0.140014,0.666006,0.325181,0.0,0.0,0.099504,0.572471,0.0,0.817143,0.237562,0.926187,0.218002,0.510484,0.099504
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,14.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,20.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,27.000000,4.000000,5.000000,1.000000,3.000000,1.000000,0.0,0.0,1.000000,4.000000,0.0,7.000000,1.000000,8.000000,1.000000,4.000000,1.000000


In [77]:
pd.DataFrame(y_train).describe()

,0
count,101.000000
mean,9.991089
std,4.512629
min,3.400000
25%,6.700000
50%,8.900000
75%,11.900000
max,23.400000


In [78]:
from sklearn.preprocessing import StandardScaler
# Skalowanie danych liczbowych aby dane o Dużych wartościach nie zostały uznane za bardziej znaczące dla modelu
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [79]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,1.010000e+02,1.010000e+02,1.010000e+02,1.010000e+02,1.010000e+02,1.010000e+02,101.0,101.0,1.010000e+02,1.010000e+02,101.0,1.010000e+02,1.010000e+02,1.010000e+02,1.010000e+02,1.010000e+02,1.010000e+02
mean,3.517538e-17,2.198461e-17,-1.319077e-17,2.803038e-17,-4.396923e-17,2.748077e-17,0.0,0.0,7.227442e-17,2.198461e-17,0.0,-4.726692e-17,-4.836615e-17,4.067154e-17,7.145000e-18,-1.297092e-16,1.319077e-17
std,1.004988e+00,1.004988e+00,1.004988e+00,1.004988e+00,1.004988e+00,1.004988e+00,0.0,0.0,1.004988e+00,1.004988e+00,0.0,1.004988e+00,1.004988e+00,1.004988e+00,1.004988e+00,1.004988e+00,1.004988e+00
min,-1.641896e+00,-5.086908e-01,-7.803750e-01,-1.421338e-01,-6.125547e-01,-3.671940e-01,0.0,0.0,-1.000000e-01,-2.607217e-01,0.0,-1.826554e-01,-2.513123e-01,-2.900712e-01,-2.282177e-01,-2.728884e-01,-1.000000e-01
25%,-8.699882e-01,-5.086908e-01,-7.803750e-01,-1.421338e-01,-6.125547e-01,-3.671940e-01,0.0,0.0,-1.000000e-01,-2.607217e-01,0.0,-1.826554e-01,-2.513123e-01,-2.900712e-01,-2.282177e-01,-2.728884e-01,-1.000000e-01
50%,3.057059e-02,-5.086908e-01,1.468941e-01,-1.421338e-01,-6.125547e-01,-3.671940e-01,0.0,0.0,-1.000000e-01,-2.607217e-01,0.0,-1.826554e-01,-2.513123e-01,-2.900712e-01,-2.282177e-01,-2.728884e-01,-1.000000e-01
75%,8.024781e-01,9.592456e-01,1.468941e-01,-1.421338e-01,8.964215e-01,-3.671940e-01,0.0,0.0,-1.000000e-01,-2.607217e-01,0.0,-1.826554e-01,-2.513123e-01,-2.900712e-01,-2.282177e-01,-2.728884e-01,-1.000000e-01
max,1.703037e+00,5.363055e+00,3.855971e+00,7.035624e+00,3.914374e+00,2.723356e+00,0.0,0.0,1.000000e+01,6.761384e+00,0.0,8.426502e+00,3.979112e+00,8.390577e+00,4.381780e+00,7.601891e+00,1.000000e+01


In [80]:
# Importowanie modelu Regresji Liniowej oraz Tablicy Pomyłek
from sklearn import linear_model
classifier = linear_model.LinearRegression()
model = classifier.fit(X_train, y_train)
classifier.intercept_, classifier.coef_
classifier.score(X_test, y_test)

0.45562475906655664

In [81]:
from sklearn import metrics
y_pred = classifier.predict(X_test)
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df

,Actual,Predicted
0,11.0,8.793671
1,7.9,8.067712
2,9.3,10.355984
3,11.3,12.023333
4,5.7,4.480716
5,7.6,7.810348
6,15.6,13.229221
7,18.3,12.967806
8,9.9,12.623976
9,9.0,7.931781


In [82]:
metrics.mean_squared_error(y_test, y_pred)

8.217094678086095

In [85]:
pd.DataFrame(y_train).shape()

TypeError: 'tuple' object is not callable

In [112]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(100,80,5), alpha=1e-3, activation = 'relu',solver='sgd',random_state=0)
mlp.fit(X_train, y_train)
mlp.score(X_test, y_test)

/home/opalkonrad/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/opalkonrad/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.41665438610972794

In [99]:
mlp_y_pred = mlp.predict(X_test)
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': mlp_y_pred.flatten()})
df

,Actual,Predicted
0,11.0,7.674733
1,7.9,6.632912
2,9.3,12.661252
3,11.3,14.469563
4,5.7,5.177441
5,7.6,7.572617
6,15.6,15.962391
7,18.3,12.600194
8,9.9,11.813381
9,9.0,6.879076
